In [ ]:
# Copyright (c) 2020-2021 Adrian Georg Herrmann

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import interpolate
from sklearn.linear_model import LinearRegression

from datetime import datetime

In [ ]:
data_root = "../../data"

locations = {
    "berlin": ["52.4652025", "13.3412466"],
    "wijchen": ["51.8235504", "5.7329005"]
}
dfs = { "berlin": None, "wijchen": None }

## Sunlight angles

In [ ]:
def get_julian_day(time):
    if time.month > 2:
        y = time.year
        m = time.month
    else:
        y = time.year - 1
        m = time.month + 12
    d = time.day + time.hour / 24 + time.minute / 1440 + time.second / 86400
    b = 2 - np.floor(y / 100) + np.floor(y / 400)

    jd = np.floor(365.25 * (y + 4716)) + np.floor(30.6001 * (m + 1)) + d + b - 1524.5
    return jd

In [ ]:
def get_angle(time, latitude, longitude):
    # Source: 
    # https://de.wikipedia.org/wiki/Sonnenstand#Genauere_Ermittlung_des_Sonnenstandes_f%C3%BCr_einen_Zeitpunkt

    # 1. Eclipctical coordinates of the sun

    # Julian day
    jd = get_julian_day(time)

    n = jd - 2451545

    # Median ecliptic longitude of the sun<
    l = np.mod(280.46 + 0.9856474 * n, 360)

    # Median anomaly
    g = np.mod(357.528 + 0.9856003 * n, 360)

    # Ecliptic longitude of the sun
    lbd = l + 1.915 * np.sin(np.radians(g)) + 0.01997 * np.sin(np.radians(2*g))


    # 2. Equatorial coordinates of the sun

    # Ecliptic
    eps = 23.439 - 0.0000004 * n

    # Right ascension
    alpha = np.degrees(np.arctan(np.cos(np.radians(eps)) * np.tan(np.radians(lbd))))
    if np.cos(np.radians(lbd)) < 0:
        alpha += 180

    # Declination
    delta = np.degrees(np.arcsin(np.sin(np.radians(eps)) * np.sin(np.radians(lbd))))


    # 3. Horizontal coordinates of the sun

    t0 = (get_julian_day(time.replace(hour=0, minute=0, second=0)) - 2451545) / 36525

    # Median sidereal time
    theta_hg = np.mod(6.697376 + 2400.05134 * t0 + 1.002738 * (time.hour + time.minute / 60), 24)

    theta_g = theta_hg * 15
    theta = theta_g + longitude

    # Hour angle of the sun
    tau = theta - alpha

    # Elevation angle
    h = np.cos(np.radians(delta)) * np.cos(np.radians(tau)) * np.cos(np.radians(latitude))
    h += np.sin(np.radians(delta)) * np.sin(np.radians(latitude))
    h = np.degrees(np.arcsin(h))

    return (h if h > 0 else 0)

## Energy data

In [ ]:
for location, _ in locations.items():
    # This list contains all time points for which energy measurements exist, therefore delimiting
    # the time frame that is to our interest.
    energy = {}

    data_path = os.path.join(data_root, location)
    for filename in os.listdir(data_path):
        with open(os.path.join(data_path, filename), "r") as file:
            for line in file:
                key = datetime.strptime(line.split(";")[0], '%Y-%m-%d %H:%M:%S').timestamp()
                energy[key] = int(line.split(";")[1].strip())

    df = pd.DataFrame(
        data={"time": energy.keys(), "energy": energy.values()},
        columns=["time", "energy"]
    )
    dfs[location] = df.sort_values(by="time", ascending=True)

In [ ]:
# Summarize energy data per hour instead of keeping it per 15 minutes

for location, _ in locations.items():
    times = []
    energy = []

    df = dfs[location]
    for i, row in dfs[location].iterrows():
        if row["time"] % 3600 == 0:
            try:
                t4 = row["time"]
                e4 = row["energy"]
                e3 = df["energy"][df["time"] == t4 - 900].values[0]
                e2 = df["energy"][df["time"] == t4 - 1800].values[0]
                e1 = df["energy"][df["time"] == t4 - 2700].values[0]

                times += [t4]
                energy += [e1 + e2 + e3 + e4]
            except:
                pass

    df = pd.DataFrame(data={"time": times, "energy_h": energy}, columns=["time", "energy_h"])
    df = df.sort_values(by="time", ascending=True)
    dfs[location] = dfs[location].join(df.set_index("time"), on="time", how="right").drop("energy", axis=1)
    dfs[location].rename(columns={"energy_h": "energy"}, inplace=True)

In [ ]:
# These lists contain the time tuples that delimit connected ranges without interruptions.
time_delimiters = {}

for location, _ in locations.items():
    delimiters = []
    df = dfs[location]

    next_couple = [df["time"].iloc[0], None]
    interval = df["time"].iloc[1] - df["time"].iloc[0]
    for i in range(len(df["time"].index) - 1):
        if df["time"].iloc[i+1] - df["time"].iloc[i] > interval:
            next_couple[1] = df["time"].iloc[i]
            delimiters += [next_couple]
            next_couple = [df["time"].iloc[i+1], None]
    next_couple[1] = df["time"].iloc[-1]
    delimiters += [next_couple]

    time_delimiters[location] = delimiters

In [ ]:
# This are lists of dataframes containing connected ranges without interruptions.

dataframes_wijchen = []
for x in time_delimiters["wijchen"]:
    dataframes_wijchen += [dfs["wijchen"].loc[(dfs["wijchen"].time >= x[0]) & (dfs["wijchen"].time <= x[1])]]

dataframes_berlin = []
for x in time_delimiters["berlin"]:
    dataframes_berlin += [dfs["berlin"].loc[(dfs["berlin"].time >= x[0]) & (dfs["berlin"].time <= x[1])]]

In [ ]:
for location, _ in locations.items():
    print(location, ":")
    for delimiters in time_delimiters[location]:
        t0 = datetime.fromtimestamp(delimiters[0])
        t1 = datetime.fromtimestamp(delimiters[1])
        print(t0, "-", t1)
    print()

### Wijchen dataset

In [ ]:
for d in dataframes_wijchen:
    print(len(d))

In [ ]:
plt.figure(figsize=(200, 25))
plt.plot(dfs["wijchen"]["time"], dfs["wijchen"]["energy"], drawstyle="steps-pre")

In [ ]:
energy_max_wijchen = dfs["wijchen"]["energy"].max()
energy_max_wijchen_idx = dfs["wijchen"]["energy"].argmax()
energy_max_wijchen_time = datetime.fromtimestamp(dfs["wijchen"]["time"].iloc[energy_max_wijchen_idx])

print(energy_max_wijchen_time, ":", energy_max_wijchen)

In [ ]:
energy_avg_wijchen = dfs["wijchen"]["energy"].mean()
print(energy_avg_wijchen)

### Berlin dataset

In [ ]:
for d in dataframes_berlin:
    print(len(d))

In [ ]:
plt.figure(figsize=(200, 25))
plt.plot(dfs["berlin"]["time"], dfs["berlin"]["energy"], drawstyle="steps-pre")

In [ ]:
energy_max_berlin = dfs["berlin"]["energy"].max()
energy_max_berlin_idx = dfs["berlin"]["energy"].argmax()
energy_max_berlin_time = datetime.fromtimestamp(dfs["berlin"]["time"].iloc[energy_max_berlin_idx])

print(energy_max_berlin_time, ":", energy_max_berlin)

In [ ]:
energy_avg_berlin = dfs["berlin"]["energy"].mean()
print(energy_avg_berlin)

## Sunlight angles

In [ ]:
for location, lonlat in locations.items():
    angles = [
        get_angle(
            datetime.fromtimestamp(x - 3600), float(lonlat[0]), float(lonlat[1])
        ) for x in dfs[location]["time"]
    ]
    dfs[location]["angles"] = angles

## Weather data

In [ ]:
# Contact the author for a sample of data, see doc/thesis.pdf, page 72.
weather_data = np.load(os.path.join(data_root, "weather.npy"), allow_pickle=True).item()

In [ ]:
# There is no cloud cover data for berlin2, so use the data of berlin1.
weather_data["berlin2"]["cloud"] = weather_data["berlin1"]["cloud"]

In [ ]:
# There is no radiation data for berlin1, so use the data of berlin2.
weather_data["berlin1"]["rad"] = weather_data["berlin2"]["rad"]

In [ ]:
# Preprocess weather data
weather_params = [ "temp", "humid", "press", "cloud", "rad" ]
stations = [ "wijchen1", "wijchen2", "berlin1", "berlin2" ]

for station in stations:
    for param in weather_params:
        to_del = []
        for key, val in weather_data[station][param].items():
            if val is None:
                to_del.append(key)
        for x in to_del:
            del weather_data[station][param][x]

In [ ]:
def interpolate_map(map, time_range):
    ret = {
        "time": [],
        "value": []
    }
    keys = list(map.keys())
    values = list(map.values())
    f = interpolate.interp1d(keys, values)
    ret["time"] = time_range
    ret["value"] = f(ret["time"])
    return ret

In [ ]:
def update_df(df, time_range, map1, map2, param1, param2):
    map1_ = interpolate_map(map1, time_range)
    df1 = pd.DataFrame(
        data={"time": map1_["time"], param1: map1_["value"]},
        columns=["time", param1]
    )

    map2_ = interpolate_map(map2, time_range)
    df2 = pd.DataFrame(
        data={"time": map2_["time"], param2: map2_["value"]},
        columns=["time", param2]
    )

    df_ = df.join(df1.set_index("time"), on="time").join(df2.set_index("time"), on="time")
    return df_

In [ ]:
# Insert weather data into dataframes
for location, _ in locations.items():
    df = dfs[location]
    station1 = location + "1"
    station2 = location + "2"

    for param in weather_params:
        param1 = param + "1"
        param2 = param + "2"
        df = update_df(
            df, df["time"], weather_data[station1][param], weather_data[station2][param], param1, param2
        )

    dfs[location] = df.set_index(keys=["time"], drop=False)

In [ ]:
# These are lists of dataframes containing connected ranges without interruptions.

dataframes_wijchen = []
for x in time_delimiters["wijchen"]:
    dataframes_wijchen += [dfs["wijchen"].loc[(dfs["wijchen"].time >= x[0]) & (dfs["wijchen"].time <= x[1])]]

dataframes_berlin = []
for x in time_delimiters["berlin"]:
    dataframes_berlin += [dfs["berlin"].loc[(dfs["berlin"].time >= x[0]) & (dfs["berlin"].time <= x[1])]]

### Linear regression model

#### Wijchen

In [ ]:
df_train = dataframes_wijchen[9].iloc[17:258]
# df_train = dataframes_wijchen[9].iloc[17:234]
# df_train = pd.concat([dataframes_wijchen[9].iloc[17:], dataframes_wijchen[10], dataframes_wijchen[11]])

df_val = dataframes_wijchen[-3].iloc[:241]
# df_val = dataframes_wijchen[-2].iloc[:241]

In [ ]:
lr_x1 = df_train[["angles", "temp1", "humid1", "press1", "cloud1", "rad1"]].to_numpy()
lr_y1 = df_train[["energy"]].to_numpy()

lr_model1 = LinearRegression()
lr_model1.fit(lr_x1, lr_y1)
lr_model1.score(lr_x1, lr_y1)

In [ ]:
lr_x2 = df_train[["angles", "temp2", "humid2", "press2", "cloud2", "rad2"]].to_numpy()
lr_y2 = df_train[["energy"]].to_numpy()

lr_model2 = LinearRegression()
lr_model2.fit(lr_x2, lr_y2)
lr_model2.score(lr_x2, lr_y2)

In [ ]:
lr_x3 = df_train[["angles", "temp1", "temp2", "humid1", "humid2", "press1", "press2", "cloud1", "cloud2", "rad1", "rad2"]].to_numpy()
lr_y3 = df_train[["energy"]].to_numpy()

lr_model3 = LinearRegression()
lr_model3.fit(lr_x3, lr_y3)
lr_model3.score(lr_x3, lr_y3)

In [ ]:
# filename = "lr_model.pkl"
# with open(filename, 'wb') as file:
#     pickle.dump(lr_model3, file)

In [ ]:
xticks = df_train["time"].iloc[::24]

lr_x3 = df_train[["angles", "temp1", "temp2", "humid1", "humid2", "press1", "press2", "cloud1", "cloud2", "rad1", "rad2"]].to_numpy()

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 5))

ax.set_xticks(ticks=xticks)
ax.set_xticklabels(labels=[datetime.fromtimestamp(x).strftime("%d-%m-%y") for x in xticks])
ax.tick_params(labelsize=18)
ax.plot(df_train["time"], df_train["energy"], label="Actual energy production in Wh", drawstyle="steps-pre")
ax.plot(df_train["time"], lr_model3.predict(lr_x3), label="Predicted energy production in Wh (Volkel + Deelen)", drawstyle="steps-pre")
ax.legend(prop={'size': 18})

In [ ]:
xticks = df_val["time"].iloc[::24]

lr_x1 = df_val[["angles", "temp1", "humid1", "press1", "cloud1", "rad1"]].to_numpy()
lr_x2 = df_val[["angles", "temp2", "humid2", "press2", "cloud2", "rad2"]].to_numpy()
lr_x3 = df_val[["angles", "temp1", "temp2", "humid1", "humid2", "press1", "press2", "cloud1", "cloud2", "rad1", "rad2"]].to_numpy()

print(lr_model1.score(lr_x1, df_val[["energy"]].to_numpy()))
print(lr_model2.score(lr_x2, df_val[["energy"]].to_numpy()))    
print(lr_model3.score(lr_x3, df_val[["energy"]].to_numpy()))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 5))

ax.set_xticks(ticks=xticks)
ax.set_xticklabels(labels=[datetime.fromtimestamp(x).strftime("%d-%m-%y") for x in xticks])
ax.tick_params(labelsize=18)
ax.plot(df_val["time"], df_val["energy"], label="Actual energy production in Wh", drawstyle="steps-pre")
ax.plot(df_val["time"], lr_model3.predict(lr_x3), label="Predicted energy production in Wh (Volkel + Deelen)", drawstyle="steps-pre")
ax.legend(prop={'size': 18})

In [ ]:
print(df["angles"].min(), df_val["angles"].max())
print(df["angles"].min(), df_train["angles"].max())

#### Berlin

In [ ]:
df_train = dataframes_berlin[1].iloc[:241]
# df_train = dataframes_berlin[1].iloc[:720]

df_val = dataframes_berlin[1].iloc[312:553]
# df_val = dataframes_berlin[1].iloc[720:961]

In [ ]:
lr_x1 = df_train[["angles", "temp1", "humid1", "press1", "cloud1", "rad1"]].to_numpy()
lr_y1 = df_train[["energy"]].to_numpy()

lr_model1 = LinearRegression()
lr_model1.fit(lr_x1, lr_y1)
lr_model1.score(lr_x1, lr_y1)

In [ ]:
lr_x2 = df_train[["angles", "temp2", "humid2", "press2", "cloud2", "rad2"]].to_numpy()
lr_y2 = df_train[["energy"]].to_numpy()

lr_model2 = LinearRegression()
lr_model2.fit(lr_x2, lr_y2)
lr_model2.score(lr_x2, lr_y2)

In [ ]:
lr_x3 = df_train[["angles", "temp1", "temp2", "humid1", "humid2", "press1", "press2", "cloud1", "cloud2", "rad1", "rad2"]].to_numpy()
lr_y3 = df_train[["energy"]].to_numpy()

lr_model3 = LinearRegression()
lr_model3.fit(lr_x3, lr_y3)
lr_model3.score(lr_x3, lr_y3)

In [ ]:
# filename = "lr_model.pkl"
# with open(filename, 'wb') as file:
#     pickle.dump(lr_model3, file)

In [ ]:
xticks = df_train["time"].iloc[::24]

lr_x3 = df_train[["angles", "temp1", "temp2", "humid1", "humid2", "press1", "press2", "cloud1", "cloud2", "rad1", "rad2"]].to_numpy()

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 5))

ax.set_xticks(ticks=xticks)
ax.set_xticklabels(labels=[datetime.fromtimestamp(x).strftime("%d-%m-%y") for x in xticks])
ax.tick_params(labelsize=18)
ax.plot(df_train["time"], df_train["energy"], label="Actual energy production in Wh", drawstyle="steps-pre")
ax.plot(df_train["time"], lr_model3.predict(lr_x3), label="Predicted energy production in Wh", drawstyle="steps-pre")
ax.legend(prop={'size': 18})

In [ ]:
xticks = df_val["time"].iloc[::24]

lr_x1 = df_val[["angles", "temp1", "humid1", "press1", "cloud1", "rad1"]].to_numpy()
lr_x2 = df_val[["angles", "temp2", "humid2", "press2", "cloud2", "rad2"]].to_numpy()
lr_x3 = df_val[["angles", "temp1", "temp2", "humid1", "humid2", "press1", "press2", "cloud1", "cloud2", "rad1", "rad2"]].to_numpy()

print(lr_model1.score(lr_x1, df_val[["energy"]].to_numpy()))
print(lr_model2.score(lr_x2, df_val[["energy"]].to_numpy()))
print(lr_model3.score(lr_x3, df_val[["energy"]].to_numpy()))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 5))

ax.set_xticks(ticks=xticks)
ax.set_xticklabels(labels=[datetime.fromtimestamp(x).strftime("%d-%m-%y") for x in xticks])
ax.tick_params(labelsize=18)
ax.plot(df_val["time"], df_val["energy"], label="Actual energy production in Wh", drawstyle="steps-pre")
ax.plot(df_val["time"], lr_model3.predict(lr_x3), label="Predicted energy production in Wh", drawstyle="steps-pre")
ax.legend(prop={'size': 18})